# Custom MicroPython

The WiFi co-processor requires two custom C modules:

* [**MessagePack**](https://msgpack.org/) is an efficient binary serializer available for many languages, and
* **FinaliserProxy**, a helper class for MicroPython to call the finaliser of user-defined Python objects.

Let's add them to the stock MicroPython interpreter.

## Install & Compile

Copy the code to ~/micropython_modules. Then run the commands below to create custom MicroPython interpreters with these features added.

I do the development on an ESP32. The code is port agnostic and can be compiled for other ports and boards as well.

In [1]:
%connect esp32

Connected to esp32 @ serial:///dev/ttyUSB0


Update local MicroPython branch:

In [1]:
%%bash

cd ~/micropython
git checkout master
git pull
git merge master

Already on 'master'
Your branch is up to date with 'origin/master'.
From https://github.com/micropython/micropython
   e3291e180..71722c84c  master     -> origin/master
Updating e3291e180..71722c84c
Fast-forward
 .github/workflows/ports_unix.yml                   |  12 +
 docs/esp32/quickref.rst                            |   2 +-
 docs/esp8266/quickref.rst                          |   6 +-
 docs/library/machine.I2C.rst                       |  38 +-
 docs/library/machine.I2S.rst                       |   2 +-
 docs/library/machine.SPI.rst                       |  14 +-
 docs/library/pyb.I2C.rst                           |  54 +-
 docs/library/pyb.SPI.rst                           |  20 +-
 docs/library/uasyncio.rst                          |   8 +
 docs/library/utime.rst                             |   9 +
 docs/pyboard/quickref.rst                          |  12 +-
 docs/pyboard/tutorial/amp_skin.rst                 |   2 +-
 docs/pyboard/tutorial/lcd_skin.rst                 |   6 +

Clone the github repo with the custom code:

In [ ]:
%%bash

cd
git clone https://github.com/iot49/micropython_modules.git

Find the port of the connected microcontroller (for flashing):

In [1]:
%info

name            esp32
platform        esp32
implementation  micropython
uid             30:ae:a4:30:84:34
url             serial:///dev/ttyUSB0
configuration   example.yaml


Compile and flash:

In [1]:
%%service esp-idf

cd ~/micropython/ports/esp32
# make submodules
# make clean 
make BOARD=GENERIC_OTA \
     USER_C_MODULES=../../../../micropython_modules/micropython.cmake \
     PORT=/dev/ttyUSB0 deploy

setting up IDF ...
idf.py -D MICROPY_BOARD=GENERIC_OTA -B build-GENERIC_OTA  -DUSER_C_MODULES=../../../../micropython_modules/micropython.cmake -p /dev/ttyUSB0 -b 460800 flash
-- Found Git: /usr/bin/git (found version "2.20.1") 
-- Component directory /opt/esp/idf/components/cmock does not contain a CMakeLists.txt file. No component will be added
-- Component directory /opt/esp/idf/components/esp_phy does not contain a CMakeLists.txt file. No component will be added
-- Component directory /opt/esp/idf/components/ieee802154 does not contain a CMakeLists.txt file. No component will be added
-- Component directory /opt/esp/idf/components/openthread does not contain a CMakeLists.txt file. No component will be added
-- IDF_TARGET not set, using default target: esp32
-- ccache will be used for faster recompilation
-- The C compiler identification is GNU 8.4.0
-- The CXX compiler identification is GNU 8.4.0
-- The ASM compiler identification is GNU
-- Found assembler: /opt/esp/tools/xtensa-es

Including User C Module(s) from ../../../../micropython_modules/micropython.cmake
Found User C Module(s): usermod_msgpack, usermod_finaliserproxy


-- Found Python3: /opt/esp/python_env/idf4.2_py3.7_env/bin/python3.7 (found version "3.7.3") found components: Interpreter 
-- Components: app_trace app_update asio bootloader bootloader_support bt cbor coap console cxx driver efuse esp-tls esp32 esp_adc_cal esp_common esp_eth esp_event esp_gdbstub esp_hid esp_http_client esp_http_server esp_https_ota esp_https_server esp_ipc esp_local_ctrl esp_netif esp_ringbuf esp_rom esp_serial_slave_link esp_system esp_timer esp_websocket_client esp_wifi espcoredump esptool_py expat fatfs freemodbus freertos heap idf_test jsmn json libsodium log lwip main mbedtls mdns mqtt newlib nghttp nvs_flash openssl partition_table perfmon protobuf-c protocomm pthread sdmmc soc spi_flash spiffs tcp_transport tcpip_adapter tinyusb ulp unity vfs wear_levelling wifi_provisioning wpa_supplicant xtensa
-- Component paths: /opt/esp/idf/components/app_trace /opt/esp/idf/components/app_update /opt/esp/idf/components/asio /opt/esp/idf/components/bootloader /opt/esp/idf

## MessagePack

An example ... note that the packed version is quite a bit smaller than the text representation. Msgpack also handles more types than json, for example bytes.

In [1]:
import msgpack, json
from io import BytesIO

# msgpack takes a "stream-like" target
buf = BytesIO()

# object to pack
obj = {'list': [True, False, None, 1, 'abc'], 'str': 'blah', 'bytes': b'012345'}

msgpack.pack(obj, buf)
print("packed   [{}]: {}".format(len(buf.getvalue()), buf.getvalue()))

# unpack
buf.seek(0)
obj_unpacked = msgpack.unpack(buf)
print("unpacked [{}]: {}".format(len(repr(obj)), obj_unpacked))

packed   [38]: b'\x83\xa3str\xa4blah\xa5bytes\xc4\x06012345\xa4list\x95\xc3\xc2\xc0\x01\xa3abc'
unpacked [74]: {'bytes': b'012345', 'list': [True, False, None, 1, 'abc'], 'str': 'blah'}


A more sophisticated example that makes use of MessagePack's custom types to serialize `MyClass`:

In [1]:
from msgpack import pack, unpack, ExtType
from io import BytesIO

class MyClass:
    def __init__(self, val):
        self.value = val
    def __str__(self):
        return str(self.value)

def encoder(obj):
    if isinstance(obj, MyClass):
        # represent MyClass as object with code == 1
        # valid codes are in the range 0~127
        return ExtType(1, obj.value)
    return "no encoder for {}".format(obj)

def decoder(code, data):
    if code == 1:
        # code == 1 is our code for MyClass
        return MyClass(data)
    return "no decoder for type {}".format(code)

data = MyClass(b'my_value')

buffer = BytesIO()
pack(data, buffer, default=encoder)
buffer.seek(0)
decoded = unpack(buffer, ext_hook=decoder)
print("{} -> {} -> {}".format(data, buffer.getvalue(), decoded))

b'my_value' -> b'\xd7\x01my_value' -> b'my_value'


RPC uses the encoder on the server and the decoder on the client to send "proxies" of classes (e.g. instances of socket) from the esp32 to the client.

## FinaliserProxy

CPython calls `__del__` when the garbage collector sweeps an object. MicroPython implements this feature only for classes written in C.

`FinaliserProxy` is a special class written in C that calls `__del__` of derived classes.

In [1]:
try:
    from finaliserproxy import FinaliserProxy
except ImportError:
    # CPython compatibility (and ports that do not implement this feature)
    print("no finaliser proxy ...")
    class FinaliserProxy:
        def __init__(self, cb):
            pass
    
import gc

class FP(FinaliserProxy):
    def __init__(self, desc):
        self.desc = desc
        super().__init__(self.__del__)
    def __del__(self):
        print("__del__: finalise", self.desc)

for i in range(20):
    f = FP("obj {}".format(i))
    if i % 4 == 0:
        print("--------- collect ...", i)
        gc.collect()

gc.collect()

print("DONE")

--------- collect ... 0
finalise obj 19
--------- collect ... 4
__del__: finalise obj 1
__del__: finalise obj 2
__del__: finalise obj 0
__del__: finalise obj 3
--------- collect ... 8
__del__: finalise obj 5
__del__: finalise obj 6
__del__: finalise obj 7
__del__: finalise obj 4
--------- collect ... 12
__del__: finalise obj 9
__del__: finalise obj 10
__del__: finalise obj 11
__del__: finalise obj 8
--------- collect ... 16
__del__: finalise obj 13
__del__: finalise obj 14
__del__: finalise obj 15
__del__: finalise obj 12
__del__: finalise obj 17
__del__: finalise obj 18
__del__: finalise obj 16
DONE


RPC uses this feature to collect the object on the server when the proxy on the client is collected.